In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv
import os

In [2]:
# URL du site web
url = 'https://www.esb.tn/programmes/masters-professionnels/master-en-marketing-digital/'
# Charger le contenu HTML depuis la page web
response = requests.get(url)
response.raise_for_status()  # Vérifier que la requête s'est bien passée
html = response.text
soup = BeautifulSoup(html, 'html.parser')

In [3]:
# Extraction des sections Objectifs, Contenu et Compétences
# Extraire la section "Objectifs"
objectifs_section = soup.find('h3', text="Objectifs")
if objectifs_section:
    objectifs = objectifs_section.find_next('p').text.strip()  # Premier paragraphe
    objectifs_suite = objectifs_section.find_next('p').find_next('p').text.strip()  # Deuxième paragraphe
    objectifs = f"{objectifs}\n{objectifs_suite}"

    # Rechercher la section "Contenu" en ignorant les espaces insécables
    contenu_section = soup.find('h3', text=lambda t: t and "Contenu" in t)
    if contenu_section:
        # Trouver la balise <ul> immédiatement après
        ul_tag = contenu_section.find_next('ul')
        if ul_tag:
            contenu_list = ul_tag.find_all('li')
            contenu = "\n".join([item.text.strip() for item in contenu_list])
            print("Contenu extrait :")
            
        else:
            print("Aucune balise <ul> trouvée après <h3> Contenu.")
    else:
        print("La section 'Contenu' n'a pas été trouvée.")

# Extraire la section "Compétences"
competences_section = soup.find('h3', text="Compétences")
if competences_section:
    competences_list = competences_section.find_next('ul').find_all('li')
    competences = "\n".join([item.text.strip() for item in competences_list])

# Format final pour l'affichage
result = f"Objectifs\n{objectifs}\n\nContenu\n{contenu}\n\nCompétences\n{competences}"

# Afficher le résultat final
print(result)

Contenu extrait :
Objectifs
Le Master Professionnel en Marketing Digital vise à former des experts en Marketing digital et en communication capables de définir des actions stratégiques et opérationnelles et de mettre en pratique des plans de communication online et offline.
Le Master Professionnel en Marketing Digital est disponible en double diplôme en Marketing de Luxe avec MBS et en Management de la Communication avec Sup’ de COM.

Contenu
DIGITAL : Webmarketing, E-business, E-communication, Infographie et Web Design, Web analyses, etc.
MARKETING : Marketing Stratégique / Opérationnel, Recherche Marketing, Analyse du comportement du Cyberconsommateur.
SOFT SKILLS : Séminaires, Développement Personnel, Gestion de projet, Langues, etc.
Projet Professionnel : Tages, PFE, Projets Intégrés, Visites d’entreprises, etc.

Compétences
Préparer les contenus de la communication (benchmark, veille, recueil, évaluation, etc.).
Concevoir et administrer une ligne éditoriale de publications, de con

C:\Users\sirine jnayeh\AppData\Local\Temp\ipykernel_10008\2851868875.py:3: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  objectifs_section = soup.find('h3', text="Objectifs")
C:\Users\sirine jnayeh\AppData\Local\Temp\ipykernel_10008\2851868875.py:10: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  contenu_section = soup.find('h3', text=lambda t: t and "Contenu" in t)
C:\Users\sirine jnayeh\AppData\Local\Temp\ipykernel_10008\2851868875.py:25: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  competences_section = soup.find('h3', text="Compétences")


In [4]:
# Extraire la section "Métiers"
metiers_section = soup.find('h3', text="Métiers")
if metiers_section:
    metiers_list = metiers_section.find_next('ul').find_all('li')
    metiers = "\n".join([item.text.strip() for item in metiers_list])

# Extraire la section "Secteurs d’activité"
secteurs_section = soup.find('h3', text="Secteurs d’activité")
if secteurs_section:
    secteurs_list = secteurs_section.find_next('ul').find_all('li')
    secteurs = "\n".join([item.text.strip() for item in secteurs_list])

# Extraire la section "Partenariats professionnels"
partenariats_section = soup.find('div', class_='stage')
if partenariats_section:
    partenariats_list = partenariats_section.find_all('p')
    partenariats = "\n".join([item.text.strip().replace('>', '').strip() for item in partenariats_list if item.text.strip()])

# Format final pour l'affichage
result = f"Métiers\n{metiers}\n\nSecteurs d’activité\n{secteurs}\n\nPartenariats professionnels\n{partenariats}"

# Afficher le résultat final
print(result)

Métiers
Chargé de la Communication Web : Directeur de Communication, Consultant en Communication, Social Media Manager
Expert des Médias Numériques : Chef de Projet Digital, Account Manager, Content Creator
Concepteur, Rédacteur Web : Copywriter, Veilleur Stratégique, Responsable Éditoriale
Responsable Marketing : Consultant Marketing, Data Manager, Brand Manager, e-Commerce Manager

Secteurs d’activité
Entreprises de services numériques
Sociétés de conseil
Agences de communication
Divers secteurs de la Banque & Assurance
Commerce & Commerce électronique
Télécommunications

Partenariats professionnels
Agences de communication
Sociétés de conseil
Banques et assurances
Entreprises de commerce et commerce électronique


C:\Users\sirine jnayeh\AppData\Local\Temp\ipykernel_10008\1374732328.py:2: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  metiers_section = soup.find('h3', text="Métiers")
C:\Users\sirine jnayeh\AppData\Local\Temp\ipykernel_10008\1374732328.py:8: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  secteurs_section = soup.find('h3', text="Secteurs d’activité")


In [5]:
# Extraction des matières par semestre
semestres_section = soup.find_all('div', class_='elementor-accordion-item')

# Stocker les matières par semestre
matieres_par_semestre = {}

# Variable pour contrôler l'ajout de "Période d'Harmonisation" une seule fois
periode_harmonisation_extracted = False

# Parcourir chaque section de semestre
for semestre in semestres_section:
    # Extraire le nom du semestre (ex: "Période d’Harmonisation", "Semestre 1", etc.)
    semestre_title = semestre.find('a')
    if semestre_title:
        semestre_name = semestre_title.text.strip()

        # Extraire les matières pour ce semestre
        contenu_section = semestre.find('div', class_='elementor-tab-content')
        if contenu_section:
            matieres_list = contenu_section.find_all('li')
            matieres = [item.text.strip() for item in matieres_list]

            # Gérer "Période d'Harmonisation" et éviter sa répétition
            if semestre_name == "Période d’Harmonisation" and not periode_harmonisation_extracted:
                matieres_par_semestre["Période d’Harmonisation"] = matieres
                periode_harmonisation_extracted = True
            elif semestre_name != "Période d’Harmonisation":
                matieres_par_semestre[semestre_name] = matieres

# Format final pour l'affichage
result = "Matières par semestre\n"

# Ajouter la "Période d’Harmonisation" en premier, s'il existe
if "Période d’Harmonisation" in matieres_par_semestre:
    result += f"\nPériode d’Harmonisation\n"
    result += "\n".join(matieres_par_semestre["Période d’Harmonisation"]) + "\n"

# Ajouter les semestres dans l'ordre
for semestre, matieres in matieres_par_semestre.items():
    if semestre != "Période d’Harmonisation":
        result += f"\n{semestre}\n"
        result += "\n".join(matieres) + "\n"

# Afficher le résultat final
print(result)

Matières par semestre

Période d’Harmonisation
Probabilité et statistique
Analyse statistique
Analyse comportement opérationnel
Atelier de programmation (R-Python)

Période d'Harmonisation
Probabilité et Statistique
Analyse Statistique
Analyse Comportement du Consommateur
Marketing Stratégique Opérationnel
Atelier de Programmation (R-Python)

Semestre 1
Management stratégique et transition digitale
Management de projet : Fondement technique
Communication digitale
Recherche marketing
e-Business
Anglais I
Efficacité professionnelle et développement personnel
Séminaire métiers I
Droit du numérique
DD & RSE
Projet intégré I

Semestre 2
Web marketing
Marketing mobile
Technologie Web
Infographie et Web design
Social media marketing
e-CRM
Management des systèmes d’information
Anglais II
Compétences personnelles et relationnelles
Séminaire métiers II
Projet intégré II
Développement Web
Introduction au Big Data

Semestre 3
Web analyses
Analyse de données marketing
e-Merchandising
e-Commerce
Pol

In [6]:
# Extraction des sections

# Objectifs
objectifs_section = soup.find('h3', text="Objectifs")
objectifs = ""
if objectifs_section:
    objectifs = objectifs_section.find_next('p').text.strip()  # Premier paragraphe
    objectifs_suite = objectifs_section.find_next('p').find_next('p').text.strip()  # Deuxième paragraphe
    objectifs = f"{objectifs}\n{objectifs_suite}"

# Contenu
contenu_section = soup.find('h3', text="Contenu")
contenu = ""
if contenu_section:
    contenu_list = contenu_section.find_next('ul').find_all('li')
    contenu = "\n".join([item.text.strip() for item in contenu_list])

# Compétences
competences_section = soup.find('h3', text="Compétences")
competences = ""
if competences_section:
    competences_list = competences_section.find_next('ul').find_all('li')
    competences = "\n".join([item.text.strip() for item in competences_list])

# Métiers
metiers_section = soup.find('h3', text="Métiers")
metiers = ""
if metiers_section:
    metiers_list = metiers_section.find_next('ul').find_all('li')
    metiers = "\n".join([item.text.strip() for item in metiers_list])

# Secteurs d'activité
secteurs_section = soup.find('h3', text="Secteurs d’activité")
secteurs = ""
if secteurs_section:
    secteurs_list = secteurs_section.find_next('ul').find_all('li')
    secteurs = "\n".join([item.text.strip() for item in secteurs_list])

# Partenariats professionnels
partenariats_section = soup.find('div', class_='stage')
partenariats = ""
if partenariats_section:
    partenariats_list = partenariats_section.find_all('p')
    partenariats = "\n".join([item.text.strip().replace('>', '').strip() for item in partenariats_list if item.text.strip()])

# Matières par semestre
semestres_section = soup.find_all('div', class_='elementor-accordion-item')
matieres_par_semestre = {}
periode_harmonisation_extracted = False

for semestre in semestres_section:
    semestre_title = semestre.find('a')
    if semestre_title:
        semestre_name = semestre_title.text.strip()
        contenu_section = semestre.find('div', class_='elementor-tab-content')
        if contenu_section:
            matieres_list = contenu_section.find_all('li')
            matieres = [item.text.strip() for item in matieres_list]
            if semestre_name == "Période d’Harmonisation" and not periode_harmonisation_extracted:
                matieres_par_semestre["Période d’Harmonisation"] = matieres
                periode_harmonisation_extracted = True
            elif semestre_name != "Période d’Harmonisation":
                matieres_par_semestre[semestre_name] = matieres

# Création des données à insérer dans le CSV
data = {
    'Objectifs': objectifs,
    'Contenu': contenu,
    'Compétences': competences,
    'Métiers': metiers,
    'Secteurs d’activité': secteurs,
    'Partenariats professionnels': partenariats
}

# Ajouter les matières par semestre au dictionnaire des données
for semestre, matieres in matieres_par_semestre.items():
    data[semestre] = "\n".join(matieres)

# Sauvegarder dans un fichier CSV
with open('./masters.csv', 'a', newline='', encoding='utf-8') as csvfile:
    # Check if the file is empty
    file_is_empty = os.stat('./masters.csv').st_size == 0

    # Create a DictWriter object
    writer = csv.DictWriter(csvfile, fieldnames=data.keys())
    
    # Write the header only if the file is empty
    if file_is_empty:
        writer.writeheader()
    
    # Write the flattened result as a row
    writer.writerow(data)



C:\Users\sirine jnayeh\AppData\Local\Temp\ipykernel_10008\370224839.py:4: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  objectifs_section = soup.find('h3', text="Objectifs")
C:\Users\sirine jnayeh\AppData\Local\Temp\ipykernel_10008\370224839.py:12: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  contenu_section = soup.find('h3', text="Contenu")
C:\Users\sirine jnayeh\AppData\Local\Temp\ipykernel_10008\370224839.py:19: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  competences_section = soup.find('h3', text="Compétences")
C:\Users\sirine jnayeh\AppData\Local\Temp\ipykernel_10008\370224839.py:26: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  metiers_section = soup.find('h3', text="Métiers")
C:\Users\sirine jnayeh\AppData\Local\Temp\ipykernel_10008\370224839.py:33: Deprec